In [4]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
# go to the google home page
driver.get("http://www.google.com")

# the page is ajaxy so the title is originally this:
print(driver.title)

# find the element that's name attribute is q (the google search box)
inputElement = driver.find_element_by_name("q")

# type in the search
inputElement.send_keys("cheese!")

# submit the form (although google automatically searches now without submitting)
inputElement.submit()

try:
    # we have to wait for the page to refresh, the last thing that seems to be updated is the title
    WebDriverWait(driver, 10).until(EC.title_contains("cheese!"))

    # You should see "cheese! - Google Search"
    print(driver.title)

finally:
    driver.quit()


Google
cheese! - Google 搜尋


In [ ]:
# Locating UI Elements (WebElements)
from selenium.webdriver.common.by import By

# by id
# <div id="coolestWidgetEvah">...</div>
element = driver.find_element_by_id("coolestWidgetEvah")
# or
element = driver.find_element(by=By.ID, value="coolestWidgetEvah")


# by class
# <div class="cheese"><span>Cheddar</span></div><div class="cheese"><span>Gouda</span></div>
cheeses = driver.find_elements_by_class_name("cheese")
# or
cheeses = driver.find_elements(By.CLASS_NAME, "cheese")


# by tag name
# <input name="cheese" type="text"/>
cheese = driver.find_element_by_name("cheese")
# or
cheese = driver.find_element(By.NAME, "cheese")


# By Link Text
# <a href="http://www.google.com/search?q=cheese">cheese</a>>
cheese = driver.find_element_by_link_text("cheese")
# or
cheese = driver.find_element(By.LINK_TEXT, "cheese")


# By Partial Link Text
# <a href="http://www.google.com/search?q=cheese">search for cheese</a>>
cheese = driver.find_element_by_partial_link_text("cheese")
# or
cheese = driver.find_element(By.PARTIAL_LINK_TEXT, "cheese")


# By CSS
# <div id="food"><span class="dairy">milk</span><span class="dairy aged">cheese</span></div>
cheese = driver.find_element_by_css_selector("#food span.dairy.aged")
# or
cheese = driver.find_element(By.CSS_SELECTOR, "#food span.dairy.aged")


# By XPath
# <input type="text" name="example" />
# <INPUT type="text" name="other" />
inputs = driver.find_elements_by_xpath("//input")
# or
inputs = driver.find_elements(By.XPATH, "//input")

In [ ]:
# Using JavaScript
# Simple example on a page that has jQuery loaded:
element = driver.execute_script("return $('.cheese')[0]")
# Finding all the input elements for every label on a page:
labels = driver.find_elements_by_tag_name("label")
inputs = driver.execute_script(
    "var labels = arguments[0], inputs = []; for (var i=0; i < labels.length; i++){" +
    "inputs.push(document.getElementById(labels[i].getAttribute('for'))); } return inputs;", labels)

In [ ]:
# Getting text values
element = driver.find_element_by_id("element_id")
element.text


# User Input - Filling In Forms
select = driver.find_element_by_tag_name("select")
allOptions = select.find_elements_by_tag_name("option")
for option in allOptions:
    print "Value is: " + option.get_attribute("value")
    option.click()

# available since 2.12
from selenium.webdriver.support.ui import Select
select = Select(driver.find_element_by_tag_name("select"))
select.deselect_all()
select.select_by_visible_text("Edam")

driver.find_element_by_id("submit").click()
# If you call this on an element within a form:
element.submit() # WebDriver will walk up the DOM until it finds the enclosing form and then calls submit on that.


In [ ]:
# Moving Between Windows and Frames
driver.switch_to.window("windowName")

# <a href="somewhere.html" target="windowName">Click here to open a new window</a>
for handle in driver.window_handles:
    driver.switch_to.window(handle)

# You can also switch from frame to frame (or into iframes):
driver.switch_to.frame("frameName")


In [ ]:
# Popup Dialogs
alert = driver.switch_to.alert
# usage: alert.dismiss(), etc.


# 上一頁 下一頁
driver.forward()
driver.back()

In [ ]:
# Cookies
# Go to the correct domain
driver.get("http://www.example.com")

# Now set the cookie. Here's one for the entire domain
# the cookie name here is 'key' and its value is 'value'
driver.add_cookie({'name':'key', 'value':'value', 'path':'/'})
# additional keys that can be passed in are:
# 'domain' -> String,
# 'secure' -> Boolean,
# 'expiry' -> Milliseconds since the Epoch it should expire.

# And now output all the available cookies for the current URL
for cookie in driver.get_cookies():
    print "%s -> %s" % (cookie['name'], cookie['value'])

# You can delete cookies in 2 ways
# By name
driver.delete_cookie("CookieName")
# Or all of them
driver.delete_all_cookies()

In [ ]:
# Changing the User Agent
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", "some UA string")
driver = webdriver.Firefox(profile)


# Drag And Drop
from selenium.webdriver.common.action_chains import ActionChains
element = driver.find_element_by_name("source")
target =  driver.find_element_by_name("target")

ActionChains(driver).drag_and_drop(element, target).perform()

#Explicit and Implicit Waits

Waiting is having the automated task execution elapse a certain amount of time before continuing with the next step. You should choose to use Explicit Waits or Implicit Waits.

WARNING: Do not mix implicit and explicit waits. Doing so can cause unpredictable wait times. For example setting an implicit wait of 10 seconds and an explicit wait of 15 seconds, could cause a timeout to occur after 20 seconds.

In [ ]:
# Explicit Waits
ff = webdriver.Firefox()
ff.get("http://somedomain/url_that_delays_loading")
try:
    element = WebDriverWait(ff, 10).until(EC.presence_of_element_located((By.ID, "myDynamicElement")))
finally:
    ff.quit()


# Expected Conditions
from selenium.webdriver.support import expected_conditions as EC
wait = WebDriverWait(driver, 10)
element = wait.until(EC.element_to_be_clickable((By.ID,'someid')))

# ---
# Implicit Waits
ff = webdriver.Firefox()
ff.implicitly_wait(10) # seconds
ff.get("http://somedomain/url_that_delays_loading")
myDynamicElement = ff.find_element_by_id("myDynamicElement")

In [ ]:
# RemoteWebDriver

from selenium import webdriver

driver = webdriver.Remote("http://localhost:4444/wd/hub", webdriver.DesiredCapabilities.FIREFOX.copy())
driver.get("http://www.google.com")

# Taking a Screenshot
driver.get_screenshot_as_file('/Screenshots/google.png')

# Using ChromeOptions
options = webdriver.ChromeOptions()
# set some options
driver = webdriver.Remote(desired_capabilities=options.to_capabilities())


In [ ]:
# Using a Proxy (only for firefox and IE, show firefix usage)
from selenium import webdriver
from selenium.webdriver.common.proxy import *

myProxy = "host:8080"

proxy = Proxy({
    'proxyType': ProxyType.MANUAL,
    'httpProxy': myProxy,
    'ftpProxy': myProxy,
    'sslProxy': myProxy,
    'noProxy': '' # set this value as desired
    })

driver = webdriver.Firefox(proxy=proxy)

# for remote
caps = webdriver.DesiredCapabilities.FIREFOX.copy()
proxy.add_to_capabilities(caps)

driver = webdriver.Remote(desired_capabilities=caps)